TO DO

- add an optional constraint to allow only one defender from any given team (risk mitigation)

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 11
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week2,assists_week3,assists_week4,assists_week5,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 21 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.677562,2.225854,0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,38.520866,0,3.636560
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.155374,0,2.584685
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.045020,0,1.896139
4,38.011000,3.801100,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,38.908482,2,3.042943
5,3.977562,1.325854,0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,19.920738,1,1.501133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.864949,0,2.551794
549,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.829692,0,2.798633
583,32.700000,4.087500,0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,...,NaN,NaN,1.4,2.4,7.8,2.0,6.400,41.575506,0,4.443202


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week2,assists_week3,assists_week4,assists_week5,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 21 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.677562,2.225854,0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,38.520866,0,3.636560
4,38.011000,3.801100,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,38.908482,2,3.042943
5,3.977562,1.325854,0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,...,-0.910517,1.000000,NaN,NaN,NaN,NaN,NaN,19.920738,1,1.501133
6,11.700000,1.950000,0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,20.226915,0,1.646897
7,5.692521,1.423130,0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,18.762833,0,1.582272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,7.300000,1.042857,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,18.297521,0,1.882231
461,31.581644,3.947705,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,37.342731,0,4.050445
470,11.834287,2.958572,1,NaN,1.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,26.019451,1,2.419833


In [5]:
optimize(df, 85.2, 'gameweek 12 prediction')

Status: Optimal
Cost: 84.50000000000001
Expected points: 62.205960273742676


,element_type,web_name,now_cost,games played,gameweek 12 prediction
id,,,,,
334,1,McCarthy,46,11.0,4.937744
142,2,James,59,8.0,4.993858
237,2,Alexander-Arnold,77,9.0,5.291340
343,2,Bednarek,44,8.0,4.790789
138,3,Mount,75,8.0,5.106112
233,3,Salah,130,11.0,5.639277
240,3,Jota,75,10.0,4.849370
265,3,Foden,83,7.0,4.788422
413,4,Antonio,82,10.0,5.681109


In [6]:
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 83.7
Expected points per week: 76.06307395295336


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
143,1,Mendy,62,10.0,4.306128
135,2,Chilwell,60,5.0,5.666128
142,2,James,59,8.0,5.370253
237,2,Alexander-Arnold,77,9.0,6.450990
256,2,Cancelo,65,11.0,5.850317
233,3,Salah,130,11.0,8.283313
250,3,Gündogan,72,7.0,5.350281
265,3,Foden,83,7.0,5.741995
268,3,Torres,67,4.0,6.296602


In [7]:
df = df.drop([268])
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 84.7
Expected points per week: 75.03487191944026


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
143,1,Mendy,62,10.0,4.306128
135,2,Chilwell,60,5.0,5.666128
142,2,James,59,8.0,5.370253
237,2,Alexander-Arnold,77,9.0,6.450990
256,2,Cancelo,65,11.0,5.850317
233,3,Salah,130,11.0,8.283313
250,3,Gündogan,72,7.0,5.350281
265,3,Foden,83,7.0,5.741995
177,4,Calvert-Lewin,80,3.0,8.513333


In [9]:
df = df.drop([119])
optimize(df, 85.5, 'xPoints_next10')

Status: Optimal
Cost: 85.49999999999999
Expected points per week: 59.772164297103885


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
69,1,Sánchez,46,11.0,41.128507
135,2,Chilwell,60,5.0,45.820218
142,2,James,59,8.0,48.504134
237,2,Alexander-Arnold,77,9.0,51.945715
256,2,Cancelo,65,11.0,45.942201
138,3,Mount,75,8.0,50.986203
233,3,Salah,130,11.0,55.439206
265,3,Foden,83,7.0,49.631099
420,3,Bowen,63,11.0,43.537673
